In [2]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [3]:
lr = 0.0002
gamma = 0.98

In [4]:
def main():
    env = gy.make('CartPole-v1')
    pi = Policy()
    score = 0
    
    for episode in range(2000): 
        state = env.reset()
        for t in range(501):
            prob = pi(torch.from_numpy(s).float())
            a = Categorical(prob).sample()
            state_, returns, done, info = env.step(a.item())
            pi.put_data((r,prob[a]))
            state = state_
            score += returns
            
            if done:
                break

In [6]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        self.fc1 = nn.Linear(4,128) # action의 가지 수 : 4가지
        self.fc2 = nn.Linear(128,2)
        self.opt = optim.Adam(self.parameters(), lr=lr)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0) # 각 행동에 대한 probability 반환
        return x
    
    def put_data(self,item):
        self.data.append(item)
    
    def train(self):
        total_return = 0
        for returns,prob in self.data[::-1]:
            total_return = returns + gamma*total_return
            loss = -torch.log(prob)*returns # (-) 붙이는 거 잊지말기!
            self.opt.zero_grad()
            loss.backward()
            self.opt.step()
        self.data = []